In [1]:
import pandas as pd
import numpy as np

tabel = pd.read_csv('data_latih_opsi_2.csv', sep=',', header=None)

In [2]:
#15 bit

In [3]:
individu = 10
num_of_parents = 2

In [4]:
tabel = tabel.values

In [5]:
data = tabel[:,:]
data

array([[1, 0, 0, 1, 0],
       [1, 0, 1, 1, 0],
       [1, 1, 2, 1, 0],
       [1, 2, 3, 0, 1],
       [1, 3, 0, 2, 0],
       [1, 3, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [1, 0, 0, 0, 0],
       [1, 2, 2, 1, 0],
       [1, 0, 3, 0, 1],
       [1, 2, 0, 2, 0],
       [1, 2, 1, 0, 0],
       [1, 1, 0, 1, 1],
       [1, 3, 2, 0, 1],
       [1, 1, 1, 1, 0],
       [1, 1, 3, 1, 0],
       [1, 3, 0, 0, 0],
       [1, 0, 2, 2, 0],
       [1, 1, 3, 2, 1],
       [1, 0, 2, 0, 0],
       [1, 3, 3, 0, 1],
       [1, 1, 3, 0, 0],
       [1, 2, 2, 2, 0],
       [1, 3, 2, 2, 0],
       [0, 2, 0, 2, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 1, 3, 1, 1],
       [0, 3, 2, 2, 0],
       [0, 0, 3, 1, 1],
       [0, 2, 2, 0, 0],
       [0, 1, 1, 0, 1],
       [0, 2, 1, 0, 0],
       [0, 3, 3, 0, 0],
       [0, 3, 0, 2, 1],
       [0, 2, 3, 0, 0],
       [0, 2, 0, 1, 1],
       [0, 0, 2, 1, 1],
       [0, 1, 3, 2, 1],
       [0, 2, 2, 2, 0],
       [0, 1, 2, 1, 1],
       [0, 0, 1,

In [6]:
# bla = tabel[0,:]

In [7]:
# bla

In [8]:
p = lambda f, i: (f[i] / sum(f))

In [9]:
def roulette(fit):
    r = np.random.uniform(0,1)
    indv = 0
    
    while r > 0:
        r -= p(fit,indv)
        
        indv += 1
    return indv-1

In [10]:
def fitness(dataset,rules):
    bnr = 0
    tot = len(dataset)
    for i, v in enumerate(dataset):
        if v[-1] == tree(v[:-1], rules):
            bnr += 1
    return bnr/tot

In [11]:
def oneorzero():
    return int(np.random.rand() > .5)

In [12]:
def encode(decoded):
    bit1, bit2, bit3, bit4, bit5 = np.zeros(3), np.zeros(4), np.zeros(4), np.zeros(3), np.zeros(1)
    bit1[decoded[0]] = 1
    bit2[decoded[1]] = 1
    bit3[decoded[2]] = 1
    bit4[decoded[3]] = 1
    bit5[0] = decoded[4]
    return np.concatenate([bit1, bit2, bit3, bit4, bit5], axis=0)

In [13]:
def decode(encoded):
    bits = [None] * 5
    bits[0], bits[1], bits[2], bits[3], bits[4] = encoded[:3], encoded[3:7], encoded[7:11], encoded[11:14], encoded[-1]
    
    nbits = []
    for i, v in enumerate(bits[:-1]):
        bitz = []
        for j, k in enumerate(v):
            if k == 1:
                bitz.append(j)
        nbits.append(bitz)
    nbits.append([bits[-1]])
    return nbits

In [14]:
def initializeRule():
    krom = [None] * 5
    krom[0], krom[1], krom[2], krom[3], krom[4] = np.zeros(3), np.zeros(4), np.zeros(4), np.zeros(3), np.zeros(1)
    
    for i, v in enumerate(krom[:-1]):
        while sum(krom[i]) == 0:
            for j, k in enumerate(v):
                krom[i][j] = oneorzero()
    krom[4] = np.array([oneorzero()])
    nkrom = np.concatenate(krom)
    return nkrom

In [15]:
def initialize(individu, brprule):
    population = []
    for i in range(individu):
        rules = []
        for j in range(brprule):
            rules.append(initializeRule())
        population.append(np.concatenate(rules, axis=0))
    return population

In [80]:
def mutation(individu):
    r = np.random.uniform(0,1)
    if r < 0.01: #1% chance
        pos = np.random.randint(len(individu))
        individu[pos] = 1 if individu[pos] == 0 else 0
    return individu

In [229]:
def crossover(parents):
    '''Sementara crossover yg gak dinamis, susah soalnya bikin yg dinamis wkwkwk'''
    leng = len(parents[0])
    r1 = np.random.randint(leng-2) + 1
    r2 = np.random.randint(leng-2) + 1
    while r1 == r2:
        r1 = np.random.randint(leng-2) + 1
        r2 = np.random.randint(leng-2) + 1
    mn = min(r1, r2)
    mx = max(r1, r2)
#     print(mn, mx)
    new_parents = [None] * 2
    new_parents[0] = np.concatenate([parents[0][:mn], parents[1][mn:mx], parents[0][mx:]])
    new_parents[1] = np.concatenate([parents[1][:mn], parents[0][mn:mx], parents[1][mx:]])
                                    
    return new_parents

In [17]:
def tree(test, rules):
    realdata = test
#     dtree_rules = rules

    bits = [None] * 5
    bits[0], bits[1], bits[2], bits[3], bits[4] = [], [], [], [], []
#     print(rules)
    dr = np.array_split(rules, len(rules)/15)
    dtree_rules = [decode(i) for i in dr]
#     for i, v in enumerate(dr):
#         print(v)
#         print(decode(v))
#     print(dtree_rules)
    ret = 0
    for p, l in enumerate(dtree_rules):
        asem = []
        for i, v in enumerate(l[:-1]):
            sem = []
#             print(v)
            for j, k in enumerate(v):
                
                
                sem.append(realdata[i] == k)
            v = any(sem)
            asem.append(v)
        if all(asem) == True:
            ret = l[-1][0]
            break
#     print("ret",ret)
    
    return ret
    

In [18]:
encodedData = []
for i, v in enumerate(tabel[:]):
    bitter = encode(v)
    encodedData.append(bitter)

In [19]:
encodedData

[array([0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.]),
 array([0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.]),
 array([0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.]),
 array([0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.]),
 array([0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.]),
 array([0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.]),
 array([0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.]),
 array([0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.]),
 array([0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1.]),
 array([0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.]),
 array([0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.]),
 array([0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.]),
 array([0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1.]),
 array([0., 1., 0., 

In [125]:
initializeRule()

array([0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1.])

In [126]:
indvs = initialize(individu, 50)

In [127]:
print(indvs[0][:3], indvs[0][2:6], indvs[0][6:10], indvs[0][10:13], indvs[0][-1])

[1. 0. 0.] [0. 0. 1. 1.] [1. 1. 1. 1.] [0. 0. 1.] 1.0


In [128]:
ex = decode([1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
print(ex)

[[0, 1, 2], [1, 3], [0, 1, 2, 3], [0, 1, 2], [1.0]]


In [124]:
boolean = []

In [26]:
# suhu = "suhu"
# waktu = "waktu"
# kondisi = "kondisi"
# kelembapan = "kelembapan"
# terbang = "terbang"

In [27]:
# ez = [0, 1, 2]

# # realdata = [1, 0, 0, 1, 0]
# realdata = [2, 0, 1, 0, 1 ]
# dtree_rules = [[[0, 1, 2], [1, 3], [0, 1, 2, 3], [0, 1, 2], [1.0]], [[1], [0], [0], [1], [0.0]], [[2], [0], [2], [2], [0.0]], [[0, 1, 2], [0, 1, 2, 3], [0, 1, 2, 3], [ 1], [1.0]], [[0, 1, 2], [0, 1, 2, 3], [0, 1, 2, 3], [ 0], [0.0]]]

# bits = [None]*5
# bits[0], bits[1], bits[2], bits[3], bits[4] = [], [], [], [], []

# ret = "invalid"
# for p, l in enumerate(dtree_rules):
#     asem = []
#     for i, v in enumerate(l[:-1]):
#         sem = []
#         for j, k in enumerate(v):
#             sem.append(realdata[i] == k)
#         v = any(sem)
#         asem.append(v)
#     if all(asem) == True:
#         ret = l[-1]
#         break
# print("ret",ret)

In [28]:
# any(bits[1])

In [29]:
# truth

In [30]:
# any([True,False])

In [31]:
decode([0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.])

[[1], [0], [0], [1], [0.0]]

In [32]:
decode([0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.])

[[2], [0], [1], [1], [1.0]]

In [33]:
decode([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

[[0, 1, 2], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2], [1.0]]

In [34]:
0 == 0.0

True

In [ ]:
generasi = 300

indvs = np.array(initialize(individu,50))
# print(indvs)
for gen in range(generasi):
    fitnesses = []
    for i, v in enumerate(indvs):
        fitnesses.append(fitness(data, v))

    parents = []
    for i in range(num_of_parents):
        parents.append(np.array(indvs[roulette(fitnesses)]))
    parents = np.array(parents)
    # print(parents)
#     print(parents.shape)
#     print(indvs[0:2, :])
    offsprings = np.array(crossover(parents))
    mutated_offspring = []
    for i, offspring in enumerate(offsprings):
        mutated_offspring.append(mutation(offspring))
    mutated_offspring = np.array(mutated_offspring)
#     print(parents.shape)
#     print(offsprings.shape)
#     print(mutated_offspring.shape)
    indvs[0:parents.shape[0]] = parents
    indvs[-parents.shape[0]:] = mutated_offspring
    
#     print(indvs.shape)
    n_fitnesses = [fitness(data, i) for i in indvs]
    bsf = np.max(n_fitnesses)
    idx = np.argmax([fitness(data, i) for i in indvs],axis=0)
    print('all fitnesses', n_fitnesses)
    print(bsf, idx)

68 195
all fitnesses [0.475, 0.5, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.4375, 0.5125]
0.5375 3
280 295
all fitnesses [0.5375, 0.4, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.5625, 0.4]
0.5625 8
583 621
all fitnesses [0.4, 0.475, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.4, 0.475]
0.5375 3
359 564
all fitnesses [0.4, 0.5375, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.4, 0.525]
0.5375 1
494 610
all fitnesses [0.475, 0.4, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.4875, 0.4]
0.5375 3
84 168
all fitnesses [0.475, 0.475, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.475, 0.5625]
0.5625 9
479 579
all fitnesses [0.5375, 0.475, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.5375, 0.475]
0.5375 0
267 675
all fitnesses [0.3625, 0.5375, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.3875, 0.525]
0.5375 1
360 628
all fitnesses [0.5375, 0.5375, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.5375, 0.5375]
0.5375 0
32 104
all fitnesses [0.3625, 0.5375, 0.4, 0.5375, 0.475, 0.475, 0.3625, 0.45, 0.375, 0.575]
0.

In [132]:
# indvs

[array([1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1.,
        0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0.,
        1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1.,
        0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0.,
        1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.,
        1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
        0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
        1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
        0., 

[array([0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 1

100 228


In [204]:
new_parents

[array([0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
        0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
        1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1.,
        1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
        0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
        0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
        1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0.,
        0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
        0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
        0., 

In [86]:
mutation([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

[1.0, 1.0, 1.0, 1.0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [74]:
np.random.randint(2)

1

In [226]:
xxx=[1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11., 12., 13., 14., 15., 16., 17., 18.]

In [228]:
print(xxx[:1],xxx[1:5],xxx[5:])

[1.0] [2.0, 3.0, 4.0, 5.0] [6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0]


In [138]:
new_parents = [None] * 2

In [139]:
new_parents

[None, None]

In [141]:
np.concatenate([[1], [1,2], [3,1]])

array([1, 1, 2, 3, 1])